# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
data.duplicated().sum()

0

In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Данные состоят из 5 параметров: количество звонков, суммарная длительность звонков в минутах, количество sms-сообщений, израсходованный интернет-трафик в Мб, каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Названия параметров, а также тип данных внесены без ошибок. Дубликатов, пропусков, некорректных значений нет, 
а, значит, с таблицей можно начать работать.

## Разбейте данные на выборки

In [8]:
df_train, df_2 = train_test_split(data, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_2, test_size=0.5, random_state=12345)

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Разобьем данные на выборки: обучающая выборка (60%), валидационная выборка (20%), тестовая выборка (20%)

## Исследуйте модели

### Дерево решений

In [10]:
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_result = result
        best_depth = depth

print('Модель: дерево решений')
print('Глубина дерева:', best_depth)
print('Accuracy:', best_result)

Модель: дерево решений
Глубина дерева: 3
Accuracy: 0.7853810264385692


### Случайный лес

In [11]:
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth

print('Модель: случайный лес')
print('Глубина дерева:', best_depth)
print('Количество деревьев:', best_est)
print('Accuracy:', best_result)

Модель: случайный лес
Глубина дерева: 8
Количество деревьев: 40
Accuracy: 0.8087091757387247


### Логистическая регрессия

In [12]:
model = LogisticRegression(solver='lbfgs')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print('Модель: логистическая регрессия')
print("Accuracy:", result)

Модель: логистическая регрессия
Accuracy: 0.7107309486780715


#### Вывод

Наилучший показатель качества был выявлен у модели RandomForestClassifier("Случайный лес"), на втором и третьем месте по метрике accuracy стоят DecisionTreeClassifier и LogisticRegression. Далее проверим эти модели на тестовой выборке, чтобы убедиться, что на результаты не повлияло переобучение моделей.

## Проверьте модель на тестовой выборке

In [13]:
model = RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)

print('Модель: случайный лес')
print('Глубина дерева: 8')
print('Количество деревьев: 40')
print('Accuracy:', result)

Модель: случайный лес
Глубина дерева: 8
Количество деревьев: 40
Accuracy: 0.7962674961119751


### Вывод

Показатель качества у модели RandomForestClassifier("Случайный лес") на тестовой выборке стал равным 0.7962674961119751, что близко к значению, которое мы видели на валидационной выборке.

## Заключение

Для анализа данных было выбраны модели: DecisionTreeClassifier, RandomForestClassifier, LogisticRegression. Все данные былии разделены три выборки: обучающая (60%), валидационная (20%), тестовая (20%). Наилучший показатель был у модели RandomForestClassifier (глубина дерева - 8, количество деревьев - 40, Accuracy - 0.7962674961119751)